# Matthews Model

In [3]:
import matthews.koba

In [4]:
start_step = 1 #

import sys
import math
import datetime
import itertools
from time import *

from matthews.koba.data_classes import fuel_types, fmc_cell, weather_conditions_class,  \
    boundary_conditions_class, SystemProperties, LayerState, special_values
from matthews.koba.utility import *
from matthews.koba.fmc_model_constants import *
from matthews.koba.litter_layer import LitterLayer

In [5]:
# litter_depth = sys.argv[1]
# canopy_trans = float(sys.argv[2])/100.0

litter_depth = 20
canopy_trans = 36.0/100.0

In [6]:
# BCfile = sys.argv[3]
# ICfile = sys.argv[4]
# OUTfile = sys.argv[5]

In [ ]:
n_model_layers = 2

In [7]:
def calculate(BCfile, ICfile, OUTfile):
    f = open(ICfile,'r')
    #print(f.readline()) #Header line
    text = f.readlines()
    f.close()
    
    IC = []   #create IC with a dummy first element.  The model code expects State[1] to be the first model layer, the atmsophere has index = 0.
    for i in range(0,n_model_layers+1):
        initial_state = LayerState()
        initial_state.T = 290
        initial_state.m = float(text[i].split('\t')[3])
        initial_state.l = 0
        initial_state.q = 0.0005
        IC.append(initial_state)
        
    #---Initialise model--------------------------------------------------------------

    #Default litter layer with parameter values from class definition
    litter_parameters = SystemProperties()
    litter_parameters.n_layers = 2
    litter_parameters.depth=int(litter_depth)/1000.00
    litter_parameters.Vm = litter_parameters.rhoBulk/litter_parameters.rhoLitter  #Litter volume fraction

    litter = LitterLayer(litter_parameters,IC)
    
    #---Prepare regularly spaced array of boundary conditions-------------------------

    #Read from file
    f = open(BCfile,'r')
    print(f.readline()) #Header line
    text = f.readlines()
    f.close()
    
    #Parse BCs
    BC = []
    for i in range(0,len(text)):
        inst = boundary_conditions_class()
        line = text[i].split('\t')

         #"date"	"t"	"q"	"rain"	"wind"	"cloud"	"soil"
         #2011-11-01 00:00:00	15.3325507287711	7.97204673499302	0	0.0327443434437841	8	3.57

        inst.date = int(mktime(strptime(line[0],"%Y-%m-%d %H:%M:%S"))) #Seconds since epoch

        basic_T = float(line[1])
        basic_q = float(line[2])
        basic_rain = float(line[3])
        basic_wind_ms = float(line[4])
        if (line[5]=="NA"):
            basic_solar = 0
        else:
            basic_solar = float(line[5])
        basic_soil = float(line[6])/100.0

        inst.soil_temperature = basic_T+273.15
        inst.soil_moisture = 0
        inst.air_temperature = basic_T+273.15
        inst.specific_humidity = min(basic_q/1000,q_sat(basic_T+273.15,101300))
        inst.wind_speed =  basic_wind_ms
        
        #    datet = datetime.datetime.fromtimestamp(inst.date)
        #    s_model = solar_radiation(datet,-33.7)
        #    s_model = s_model*(1-0.72*(basic_cloud/8)**3.20)
        inst.solar_radiation = basic_solar #canopy_trans*s_model
        inst.thermal_radiation = (0.9*5.67e-8*(basic_T+273.15)**4) #*(1+0.2*(basic_cloud/8))
        
        #    #Added 20140121
        #    inst.air_temperature = inst.air_temperature - inst.solar_radiation/250
        #    inst.soil_temperature = inst.soil_temperature - inst.solar_radiation/250

        inst.rain_rate = basic_rain/3600
        inst.dt = 600 #10 minute intervals
        
        BC.append(inst)
    
    #---Run model---------------------------------------------------------------------
    f = open(OUTfile,'w')
    SEP = '\t'

    start = time()
    n = len(BC)
    for i in range(start_step,n):
    #for i in range(0,n):
        BCnext = BC[i]
        litter.Step(BCnext)
        line = strftime("%Y%m%d %H:%M:%S",localtime(BC[i].date))+SEP
        print (line)
        for j in range(1,n_model_layers+1):
            line = line +'%.1f' % litter.state[j].T+SEP
        for j in range(1,n_model_layers+1):
            line = line +'%.3f' % litter.state[j].m+SEP
        for j in range(1,n_model_layers+1):
            line = line +'%.2f' % litter.state[j].l+SEP
        for j in range(1,n_model_layers+1):
            line = line +'%.4f' % litter.state[j].q+SEP
        f.write(line+'\n')
        f.flush()

    finish = time()
    print("Finished.  Time taken was "+str(finish-start)+" seconds.")
    f.close()


## Getting IC from BOM data.

Rainfall - have already from DFMC model.
precipitation_url = "http://www.bom.gov.au/web03/ncc/www/awap/rainfall/totals/daily/grid/0.05/history/nat/"

### Products
- IDV71017_VIC_Sky_SFC.nc
- IDV71015_VIC_DailyPrecip50Pct_SFC.nc
- IDV71006_VIC_Wind_Mag_SFC.nc
- IDV71000_VIC_T_SFC.nc
- IDV71018_VIC_RH_SFC.nc



In [ ]:
# NetCDF export
    